<a href="https://colab.research.google.com/github/Hagar633/Machine-learning-/blob/main/symmetry_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y mediapipe
!pip install mediapipe
!pip install decord

!pip install git+https://github.com/apple/ml-depth-pro.git
!mkdir -p checkpoints
!wget https://huggingface.co/apple/DepthPro/resolve/main/depth_pro.pt -O checkpoints/depth_pro.pt

!rm -f checkpoints/depth_pro.pt


!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118


  Cloning https://github.com/apple/ml-depth-pro.git to /tmp/pip-req-build-y527p4dy
  Running command git clone --filter=blob:none --quiet https://github.com/apple/ml-depth-pro.git /tmp/pip-req-build-y527p4dy
  Resolved https://github.com/apple/ml-depth-pro.git to commit 9efe5c1def37a26c5367a71df664b18e1306c708
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 57.7 MB/s eta 0:00:00
  Created wheel for depth_pro: filename=depth_pro-0.1-py3-none-any.whl size=27593 sha256=657863b8467ac284db44d6d7b474769977da6923abce76aeafe03ad86d1a9e7e
  Stored in directory: /tmp/pip-ephem-wheel-cache-lrwwi0w0/wheels/3f/ad/4e/a2b790772df19931cab8bf9c5af51665ae6502f39589e662c4
Successfully built depth_pro
--2025-09-02 13:10:10--  https://huggingface.co/apple/DepthPro/resolve/main/depth_pro.pt
Resolving huggingface.co (huggingface.co)... 13.35.202.121, 13.35.202.34, 13.3

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import torch
import depth_pro
model, transform = depth_pro.create_model_and_transforms()
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device).eval()
model = model.half().to(device).eval()
from pickle import FRAME
from google.colab.patches import cv2_imshow
from decord import VideoReader
import mediapipe as mp
import cv2
import numpy as np
from matplotlib import pyplot as plt
import statistics
import math


def approximate_intrinsics(width, height):
    c_x, c_y = width/2, height/2
    return float(c_x), float(c_y)


def pixel_to_camera_coords(x, y, Z, f_x, f_y, c_x, c_y):
   X = (x - c_x) * Z / f_x
   Y = (y - c_y) * Z / f_y
   return X, Y, Z

def camera_to_pixel_coords(X, Y, Z, f_x, f_y, c_x, c_y):
    u = int((X * f_x) / Z + c_x)
    v = int((Y * f_y) / Z + c_y)
    return u, v

def stability_calc(frame, depth_map, fx,fy, results, c_x, c_y):
  points = {}
  mp_pose = mp.solutions.pose

  h, w, _ = frame.shape
  for id, lm in enumerate(results.pose_landmarks.landmark):
        cx= min(max(int(lm.x * w), 0), w - 1)
        cy= min(max(int(lm.y * h), 0), h - 1)
        cz= depth_map[cy, cx]

        X, Y, Z = pixel_to_camera_coords(cx, cy, cz, fx, fy, c_x, c_y)
        if id == mp_pose.PoseLandmark.LEFT_ANKLE.value:
            points["left_ankle"] = (X, Y, Z)
        elif id == mp_pose.PoseLandmark.RIGHT_ANKLE.value:
            points["right_ankle"] = (X, Y, Z)
        elif id == mp_pose.PoseLandmark.LEFT_HIP.value:
            points["left_hip"] = (X, Y, Z)
        elif id == mp_pose.PoseLandmark.RIGHT_HIP.value:
            points["right_hip"] = (X, Y, Z)

        if id in [mp_pose.PoseLandmark.LEFT_ANKLE.value,
                  mp_pose.PoseLandmark.RIGHT_ANKLE.value]:
            cv2.circle(frame, (cx, cy), 5, (0, 255, 0), cv2.FILLED)




  com_x = (points["left_hip"][0] + points["right_hip"][0]) / 2
  com_y = (points["left_hip"][1] + points["right_hip"][1]) / 2
  com_z = (points["left_hip"][2] + points["right_hip"][2]) / 2

  u = int((com_x * fx) / com_z + c_x)
  v = int((com_y * fy) / com_z + c_y)
  # Draw COM as a red circle
  cv2.circle(frame, (u, v), 7,(0, 255, 0), cv2.FILLED)
  cv2.putText(frame,
            "COM",
            (u + 10, v - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.7,
            (255, 255, 255),
            2,
            cv2.LINE_AA)

  foot_midx = (points["left_ankle"][0] + points["right_ankle"][0]) / 2
  foot_midy = (points["left_ankle"][1] + points["right_ankle"][1]) / 2
  foot_midz = (points["left_ankle"][2] + points["right_ankle"][2]) / 2

  stance_width = math.sqrt(
    (points["left_ankle"][0] - points["right_ankle"][0])**2 +
    (points["left_ankle"][1] - points["right_ankle"][1])**2 +
    (points["left_ankle"][2] - points["right_ankle"][2])**2
   )

  sway = math.sqrt(
    (com_x - foot_midx)**2 +
    (com_y - foot_midy)**2 +
    (com_z - foot_midz)**2
  )



  norm= sway/stance_width
  u_foot = int((foot_midx * fx) / foot_midz + c_x)
  v_foot = int((foot_midy * fy) / foot_midz + c_y)

# Draw foot midpoint
  cv2.circle(frame, (u_foot, v_foot), 7, (0, 0, 255), cv2.FILLED)
  cv2.putText(frame, "Foot Mid", (u_foot + 10, v_foot - 10),
            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2, cv2.LINE_AA)
  cv2.circle(frame, (u_foot, v_foot), 7, (0, 0, 255), cv2.FILLED)


# Draw vertical line at Foot Midpoint x-position
  u_left_ankle = int((points["left_ankle"][0] * fx) / points["left_ankle"][2] + c_x)
  v_left_ankle = int((points["left_ankle"][1] * fy) / points["left_ankle"][2] + c_y)

  u_right_ankle = int((points["right_ankle"][0] * fx) / points["right_ankle"][2] + c_x)
  v_right_ankle = int((points["right_ankle"][1] * fy) / points["right_ankle"][2] + c_y)

  ankle_y = max(v_left_ankle, v_right_ankle)


  u_left_hip = int((points["left_hip"][0] * fx) / points["left_hip"][2] + c_x)
  v_left_hip = int((points["left_hip"][1] * fy) / points["left_hip"][2] + c_y)

  u_right_hip = int((points["right_hip"][0] * fx) / points["right_hip"][2] + c_x)
  v_right_hip = int((points["right_hip"][1] * fy) / points["right_hip"][2] + c_y)

  hip_y = min(v_left_hip, v_right_hip)

# Draw vertical COM line


# Draw vertical Foot Midpoint line
  cv2.line(frame, (u, v), (u_foot, v_foot), (0, 255, 255), 2)
  u_left = int((points["left_ankle"][0] * fx) / points["left_ankle"][2] + c_x)
  v_left = int((points["left_ankle"][1] * fy) / points["left_ankle"][2] + c_y)

  u_right = int((points["right_ankle"][0] * fx) / points["right_ankle"][2] + c_x)
  v_right = int((points["right_ankle"][1] * fy) / points["right_ankle"][2] + c_y)

# Draw stance width line
  cv2.line(frame, (u_left, v_left), (u_right, v_right), (255, 0, 0), 2)  # blue



# Draw stance width text near foot midpoint line
  mid_u = (u_left + u_right) // 2
  mid_v = (v_left + v_right) // 2


  return norm, sway, stance_width

# Sample every 5 seconds (assuming 30 fps)
def process_video(video_path, output_path):

  vr = VideoReader(video_path)
  frame_count = len(vr)
  print(f"Total frames: {frame_count}")
  fps = 7
  step = fps * 1
  mp_pose = mp.solutions.pose
  pose = mp_pose.Pose(static_image_mode=True)
  mp_draw = mp.solutions.drawing_utils

  sample_frame = vr[0].asnumpy()
  height, width, _ = sample_frame.shape
  c_x,c_y= approximate_intrinsics(width, height)
  fourcc = cv2.VideoWriter_fourcc(*'mp4v')
  out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
  var = 0
  norms = []
  for i in range(0, frame_count, step):

      frame = vr[i].asnumpy()
      rgb = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)
      results = pose.process(rgb)
      img = transform(rgb).unsqueeze(0).to(device).half()
      with torch.no_grad():
          pred = model.infer(img)
          depth_map = pred["depth"].squeeze().cpu().numpy()
          depth_map = cv2.resize(depth_map, (frame.shape[1], frame.shape[0]))
          intrinsics = pred["focallength_px"].cpu().numpy()
          if np.ndim(intrinsics) == 0:
            fx = fy = float(intrinsics)
          else:
            fx, fy = intrinsics

      if results.pose_landmarks:
        resultt= stability_calc(frame, depth_map, fx,fy, results,c_x,c_y)
        norm= resultt[0]
        norms.append(norm)
      if len(norms) > 1:
        var = statistics.variance(norms)
      print("stability is" , norm)
      cv2.putText(frame,
            f"Frame: {i} | Stability: {norm:.2f} | Variance: {var:.2f}",
            (10, 30),  # (x,y) for first line
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,       # bigger font
            (0, 0, 200),
            2,
            cv2.LINE_AA)

# Second line (Sway, Stance Width)
      cv2.putText(frame,
            f"Sway: {resultt[1]:.2f} | Stance Width: {resultt[2]:.2f}",
            (10, 60),  # a bit lower so it doesn’t overlap first line
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 0, 200),
            2,
            cv2.LINE_AA)
      out.write(frame)


# adding final frame
  black_frame = np.zeros_like(frame)
  avg_stability = np.mean(norms)
  final_var = statistics.variance(norms) if len(norms) > 1 else 0


  for _ in range(fps * 3):

      frame_with_text = black_frame.copy()

      txt = f"Average Stability: {avg_stability:.2f} | Final Variance: {final_var:.2f}"
      cv2.putText(frame_with_text,
                txt,
                (5, height // 2),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 255, 255),
                2,
                cv2.LINE_AA)

      out.write(frame_with_text)

  out.release()
  print(f"Video saved to {output_path}")

import os

input_folder = "/content/drive/MyDrive/test"
output_folder = "/content/drive/MyDrive/kidsfin2"
os.makedirs(output_folder, exist_ok=True)

video_files = [f for f in os.listdir(input_folder) if f.lower().endswith((".mp4", ".avi", ".mov"))]

for video in video_files:
    input_path = os.path.join(input_folder, video)
    output_name = os.path.splitext(video)[0] + "_fin.mp4"
    output_path = os.path.join(output_folder, output_name)

    print(f"Processing {video} ...")
    process_video(input_path, output_path)
    print(f"Saved result to {output_path}")




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Processing WhatsApp Video 2025-08-31 at 14.23.30_81185f5a.mp4 ...
Total frames: 481
stability is 1.6443476976274867
stability is 1.647417639002233
stability is 1.631089421695926
stability is 1.5952619336374145
stability is 1.623016201254168
stability is 1.5921399751773797
stability is 1.6290651874433757
stability is 1.5637944357175178
stability is 1.7303508487070791
stability is 1.6445984647748344
stability is 1.6997566320509219
stability is 1.82115720314338
stability is 1.6825609853287764
stability is 1.6643307760118218
stability is 1.6904399189580586
stability is 1.6061442565822868
stability is 1.7730131726274292
stability is 1.7421540046019168
stability is 1.6284160200371818
stability is 1.6365086085954426
stability is 1.6528536174568562
stability is 1.754141243133273
stability is 1.79440485581336
stability is 1.7310278804687746
stability is 1.703579140998